In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import spearmanr,pearsonr
import sys
from math import log10
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from scipy import stats
from csv import writer 

In [2]:
def get_selected_prs(target, method, thr):
    sig_prs = pd.read_csv('..\manuscript\Significant_proteins.csv')
    all_prts = pd.read_csv('..\manuscript\NAME_OID.csv')
    sig_prs_oid = all_prts[all_prts['name'].isin(sig_prs['%s_proteins'%target].values.tolist())]['OID'].values.tolist()
    high_cor_prs = sig_prs[((sig_prs['sis3_correlations'].abs()).ge(thr)) & (sig_prs['sis3_pvalue'].le(0.05))]
    high_cor_oid = all_prts[all_prts['name'].isin(high_cor_prs['%s_proteins'%target].values.tolist())]['OID'].values.tolist()
    if method == 'sig':
        return sig_prs_oid  
    if method == 'high_corr':
        return high_cor_oid 
    return 

In [3]:
def fs_func(target, data, add_fea, method, thr):
    feat1 = get_selected_prs(target, method, thr)
    feat1 = [f for f in feat1 if f in data]
    feat2 = add_fea
    k_feas = []
    k_feas.extend(feat1)
    k_feas.extend(feat2)
    return k_feas  

In [4]:
NPX_data_V2 = pd.read_csv('..\manuscript\C_NPX_data.csv')
NPX_data_V2 = NPX_data_V2.dropna(axis=1)
patient_data_v2 = pd.read_csv('..\manuscript\C_patient_data.csv')

In [8]:
target = ['facit']#['sis3'] ##
add_fea = ['age','timesince','size','gender','nihss',]#, # add or remove features
str_add_fea = 'pr +' + '+ '.join(add_fea)
#print(str_add_fea)

y_data_features = []
y_data_features.extend(add_fea)
y_data_features.extend(target)
y_data_features.extend(['pid'])
y_data = patient_data_v2.dropna(subset=target)
y_data = y_data[y_data_features] #'size','timesince','age',
y_data = y_data.dropna(subset=add_fea)

X = NPX_data_V2

data = y_data.merge(X,how='inner', on='pid')

selected_features = [col for col in NPX_data_V2 if 'OID' in col]
selected_features.extend(add_fea)
pids = set(data['pid'].values.tolist())   

In [9]:
tru_vals = []
predictions = []
fs_method = 'no filter'# 'high_corr'#   'sig' # 
thr = 0.3
x_shape = len(selected_features)
lens = []
coefs = []
inters = []
for i,p in enumerate(list(pids)):
    model = LogisticRegression(fit_intercept=True)
    if fs_method == 'no filter':
        new_selected_features = selected_features
    else:
        new_selected_features = fs_func(target[0], data, add_fea, fs_method, thr)
    x_tr = data[data['pid']!=p][selected_features].values.reshape(-1, len(selected_features))
    y_tr = data[data['pid']!=p][target].values.reshape(-1, 1)
    data2 = data.copy()
    data2[[c for c in data if ((c not in new_selected_features)) and ( c != 'pid') and (c != target[0])]]=0
    x_te = data2[data2['pid']==p][selected_features].values.reshape(-1, len(selected_features))
    y_te = data2[data2['pid']==p][target].values.reshape(-1, 1)
    tru_vals.extend(y_te.flatten())
    model.fit(x_tr, y_tr)
    y_pred = model.predict(x_te)
    predictions.extend(y_pred)

In [10]:
List = [type(model).__name__, fs_method+str_add_fea+' thr:'+ str(thr), target,  spearmanr(predictions, tru_vals)[0], spearmanr(predictions, tru_vals)[1],  -log10(spearmanr(predictions, tru_vals)[1])]
file_name = '%s.csv'%target[0]
with open('../output/%s'%file_name, 'a') as f_object: 
    writer_object = writer(f_object) 
    writer_object.writerow(List) 
    f_object.close()